## Imports
---

In [ ]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import matplotlib.pyplot as plt



import re #Regexs
import os #Acces a fitxers

## Exploració de dades
---

In [ ]:
# Obtener las categorías disponibles
categories = mr.categories()
print(categories)

# Obtener las IDs de las revisiones de películas
file_ids = mr.fileids()
print(file_ids[8])



In [ ]:
mr.raw(file_ids[0])

## Preprocessing
---

In [ ]:
import re

def preprocessing(text):
    # Tokenización
    text = re.sub(r'[^\w\s]', '', text) # Elimina signes de puntuació
    text = re.sub(r'\d+', '', text) # Elimina números
    text = text.lower() # Converteix a minúscules
    text = re.sub(r'[\s\n]+', ' ', text) # Elimina espais en blanc sobrants excepte \n
    text = re.sub(r' +', ' ', text) # Elimina espais en blanc llargs
    tokens = nltk.word_tokenize(text)
    
    return ' '.join(tokens)

In [ ]:
X, y = [], []

# Preprocesamiento de todas las reviews
for elem in file_ids:
    X.append(preprocessing(mr.raw(elem)))
    y.append(mr.categories(elem)[0])


## Train-Test division
---

In [ ]:
y_num = []

for elem in y:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num.append(elem)

In [ ]:
#dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y_num, test_size=0.2, random_state=69)

## Representació
---

- Lematitzar 
- Vectoritzar (fit en el train, transform en el train i en el test) (counts >= 5)
- Modelos


In [ ]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

def lem_func(text):
    tokens = nltk.word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [ ]:
X_train_lemmatized = []

for text in X_train:
    X_train_lemmatized.append(lem_func(text))

In [ ]:
cv = CountVectorizer(min_df=5)
cv.fit(X_train_lemmatized)

def vect(text):
    a = cv.transform(text)
    return pd.DataFrame(a.toarray(), columns=cv.get_feature_names_out())


In [ ]:
X_train_cv = vect(X_train_lemmatized)

In [ ]:
X_train_cv.info()

In [ ]:
#Eliminar stop words del dataframe